In [11]:
!pip install sktime

   ---------------------------------------- 0.0/37.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/37.0 MB 1.9 MB/s eta 0:00:20
   ---------------------------------------- 0.2/37.0 MB 2.3 MB/s eta 0:00:16
   ---------------------------------------- 0.4/37.0 MB 2.8 MB/s eta 0:00:14
   ---------------------------------------- 0.4/37.0 MB 2.9 MB/s eta 0:00:13
   ---------------------------------------- 0.4/37.0 MB 2.9 MB/s eta 0:00:13
    --------------------------------------- 0.8/37.0 MB 3.2 MB/s eta 0:00:12
   - -------------------------------------- 1.1/37.0 MB 3.8 MB/s eta 0:00:10
   - -------------------------------------- 1.3/37.0 MB 3.6 MB/s eta 0:00:10
   - -------------------------------------- 1.4/37.0 MB 3.6 MB/s eta 0:00:10
   - -------------------------------------- 1.6/37.0 MB 3.6 MB/s eta 0:00:10
   - -------------------------------------- 1.8/37.0 MB 3.7 MB/s eta 0:00:10
   -- ------------------------------------- 1.9/37.0 MB 3.7 MB/s eta 0:00:10
   -- 

In [22]:
!pip install pmdarima

   ---------------------------------------- 0.0/614.7 kB ? eta -:--:--
   - ------------------------------------- 30.7/614.7 kB 660.6 kB/s eta 0:00:01
   ----- ---------------------------------- 92.2/614.7 kB 1.1 MB/s eta 0:00:01
   -------------- ------------------------- 225.3/614.7 kB 1.7 MB/s eta 0:00:01
   ------------------- -------------------- 307.2/614.7 kB 1.9 MB/s eta 0:00:01
   ----------------------- ---------------- 368.6/614.7 kB 1.6 MB/s eta 0:00:01
   ------------------------------------ --- 563.2/614.7 kB 2.2 MB/s eta 0:00:01
   ------------------------------------- -- 583.7/614.7 kB 1.8 MB/s eta 0:00:01
   -------------------------------------- - 593.9/614.7 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 614.7/614.7 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   -- ------------------------------------- 0.2/2.8 MB 3.6 MB/s eta 0:00:01
   -- ------------------------------------- 0.2/2.8 MB 4.2 MB/s e

In [1]:
import numpy as np
import pandas as pd

from sktime.forecasting.arima import ARIMA, AutoARIMA
# from sktime.forecasting.arima import StatsModelsARIMA
# from sktime.forecasting.statsforecast import StatsForecastAutoARIMA
from sktime.utils.plotting import plot_series
# временной горизонт для прогнозирования
from sktime.forecasting.base import ForecastingHorizon

import pandas_datareader.data as web

# тесты диагностики
from statsmodels.stats.diagnostic import het_arch, acorr_ljungbox

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)
# Не показывать ValueWarning, ConvergenceWarning из statsmodels
from statsmodels.tools.sm_exceptions import ValueWarning, ConvergenceWarning
warnings.simplefilter('ignore', category=ValueWarning)
warnings.simplefilter('ignore', category=ConvergenceWarning)

## Первое задание

In [2]:
mortgage = web.DataReader(name='MORTGAGE15US', data_source='fred', start='2010-01-01', end='2024-01-31' )
mortgage

,MORTGAGE15US
DATE,
2010-01-07,4.50
2010-01-14,4.45
2010-01-21,4.40
2010-01-28,4.39
2010-02-04,4.40
...,...
2023-12-28,5.93
2024-01-04,5.89
2024-01-11,5.87


In [3]:
y=mortgage.diff().dropna()

In [4]:
r2 = y['MORTGAGE15US'].autocorr(lag=2)
r2.round(3)

0.122

## Второе задание

In [5]:
mortgage30 = web.DataReader(name='MORTGAGE30US', data_source='fred', start='2005-01-01', end='2024-01-31' )
mortgage30

,MORTGAGE30US
DATE,
2005-01-06,5.77
2005-01-13,5.74
2005-01-20,5.67
2005-01-27,5.66
2005-02-03,5.63
...,...
2023-12-28,6.61
2024-01-04,6.62
2024-01-11,6.66


In [11]:
y2=mortgage30.dropna()

In [12]:
forecaster = AutoARIMA(information_criterion='bic', test='adf', seasonal=False) #тут меняем bic и adf
forecaster.fit(y2)
forecaster.get_fitted_params() #['order']

{'sigma2': 0.009449370573161215,
 'order': (0, 1, 0),
 'seasonal_order': (0, 0, 0, 0),
 'aic': -1810.9097053205605,
 'aicc': -1810.905673062496,
 'bic': -1806.0079681139039,
 'hqic': -1809.0461590225698}

### order: 0, 1, 0 => p = 0, d= 1, q= 0

## Задание 3

In [10]:
bond = web.DataReader(name='WAAA', data_source='fred', start='2005-01-01', end='2024-01-31' )
bond

,WAAA
DATE,
2005-01-07,5.47
2005-01-14,5.40
2005-01-21,5.30
2005-01-28,5.28
2005-02-04,5.20
...,...
2023-12-29,4.63
2024-01-05,4.81
2024-01-12,4.79


In [13]:
y3=bond.dropna()

In [14]:
forecaster = ARIMA(order=(1,2,1), trend='n')
forecaster.fit(y3)
forecaster.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  995
Model:               SARIMAX(1, 2, 1)   Log Likelihood                 939.065
Date:                Fri, 11 Apr 2025   AIC                          -1872.130
Time:                        22:02:03   BIC                          -1857.428
Sample:                    01-07-2005   HQIC                         -1866.540
                         - 01-26-2024                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.1921      0.014     13.519      0.000       0.164       0.220
ma.L1         -0.9999      0.089    -11.284      0.000      -1.174      -0.826
sigma2         0.0088      0.001     11.065      0.000       0.007       0.010
===================================================================================
Ljung-Box (L1) (Q):                   0.06   Jarque-Bera (JB):              3675.41
Prob(Q):                              0.81   Prob(JB):                         0.00
Heteroskedasticity (H):               1.34   Skew:                             0.25
Prob(H) (two-sided):                  0.01   Kurtosis:                        12.41
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [15]:
forecaster.get_fitted_params()

{'ar.L1': 0.1921105427449994,
 'ma.L1': -0.9999123413660246,
 'sigma2': 0.008775201176951292,
 'aic': -1872.1298270842217,
 'aicc': -1872.1055601479225,
 'bic': -1857.4276350920861,
 'hqic': -1866.540063288201}

In [17]:
# model_df = p+q
acorr_ljungbox(forecaster.predict_residuals(y3).dropna() , lags=[8], model_df=1+1) #1+1 и указываем число лагов

,lb_stat,lb_pvalue
8,15.349518,0.017706


## p_value > уровень значимости (1%) => результат не является статистически значимым (ряд не стационарен) + НЕ отвергаем гипотезу

## Задание 4

In [18]:
bond2 = web.DataReader(name='WAAA', data_source='fred', start='2005-01-01', end='2024-01-31' )
bond2

,WAAA
DATE,
2005-01-07,5.47
2005-01-14,5.40
2005-01-21,5.30
2005-01-28,5.28
2005-02-04,5.20
...,...
2023-12-29,4.63
2024-01-05,4.81
2024-01-12,4.79


In [19]:
y4=bond.dropna()

In [22]:
forecaster = ARIMA(order=(2,1,1), trend='c')
forecaster.fit(y4)
# model_df = p+q
acorr_ljungbox(forecaster.predict_residuals(y3).dropna() , lags=[8], model_df=2+1) #1+1 и указываем число лагов (в условии не видно)

,lb_stat,lb_pvalue
8,15.765115,0.007548


## p_value < уровень значимости (1%) => результат является статистически значимым

## Задание 5

## p = 1 (y(t-1) один раз), d = 2 (степень diff), q = 1 (u(t-1) один раз)

### если было бы y(t-1) + y(t-2), то p=2 (на знаки мы не смотрим, смотрим на число лагов!)
### если был бы u(t-1) + u(t-2) + u(t-3), то q = 3 (на знаки мы не смотрим, смотрим на число лагов!)

## Задание 6

## p = 2, d=0 (нет diff), q=3

## Задание 7

In [38]:
y7 = web.DataReader(name='WAAA', data_source='fred', start='2005-01-01', end='2024-01-31')

In [41]:
forecaster = ARIMA(order=(2,1,1), trend='n')
forecaster.fit(y7)

ARIMA(order=(2, 1, 1), trend='n')

In [43]:
## ???

## Задание 8

In [44]:
mortgage8 = web.DataReader(name='MORTGAGE30US', data_source='fred', start='2005-01-01', end='2024-01-31' )
mortgage8

,MORTGAGE30US
DATE,
2005-01-06,5.77
2005-01-13,5.74
2005-01-20,5.67
2005-01-27,5.66
2005-02-03,5.63
...,...
2023-12-28,6.61
2024-01-04,6.62
2024-01-11,6.66


In [46]:
y8=mortgage8.dropna()

In [48]:
forecaster = ARIMA(order=(1,1,1), trend='c')
forecaster.fit(y8)
forecaster.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  995
Model:               SARIMAX(1, 1, 1)   Log Likelihood                 913.821
Date:                Fri, 11 Apr 2025   AIC                          -1819.643
Time:                        22:35:52   BIC                          -1800.036
Sample:                             0   HQIC                         -1812.189
                                - 995                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0003      0.001      0.213      0.831      -0.002       0.003
ar.L1          0.6917      0.106      6.542      0.000       0.484       0.899
ma.L1         -0.6037      0.115     -5.255      0.000      -0.829      -0.379
sigma2         0.0093      0.000     43.394      0.000       0.009       0.010
===================================================================================
Ljung-Box (L1) (Q):                   1.41   Jarque-Bera (JB):              1579.89
Prob(Q):                              0.23   Prob(JB):                         0.00
Heteroskedasticity (H):               1.03   Skew:                             0.57
Prob(H) (two-sided):                  0.77   Kurtosis:                         9.07
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [49]:
forecaster.get_fitted_params()

{'intercept': 0.00027563765521471903,
 'ar.L1': 0.6916572636516172,
 'ma.L1': -0.6037419336509383,
 'sigma2': 0.009309827966619819,
 'aic': -1819.642982946447,
 'aicc': -1819.602538052615,
 'bic': -1800.036034119821,
 'hqic': -1812.1887977544843}

### ar.L1: 0.6916572636516172 - это фи
### ma.L1: -0.6037419336509383 - это тета

## Задание 9

In [50]:
mortgage9 = web.DataReader(name='MORTGAGE30US', data_source='fred', start='2010-01-01', end='2024-01-31' )
mortgage9

,MORTGAGE30US
DATE,
2010-01-07,5.09
2010-01-14,5.06
2010-01-21,4.99
2010-01-28,4.98
2010-02-04,5.01
...,...
2023-12-28,6.61
2024-01-04,6.62
2024-01-11,6.66


In [51]:
y9=mortgage9.dropna()

In [54]:
forecaster = AutoARIMA(information_criterion='bic', test='kpss', seasonal=False)
forecaster.fit(y9)
forecaster.get_fitted_params() #['order']

{'ar.L1': 0.11159116182768825,
 'ar.L2': 0.11425642470719219,
 'sigma2': 0.00815813256348436,
 'order': (2, 1, 0),
 'seasonal_order': (0, 0, 0, 0),
 'aic': -1438.560550104269,
 'aicc': -1438.5276282935693,
 'bic': -1424.769112998609,
 'hqic': -1433.2407275877924}

### Что из этого является тестовой статистикой?..